Importing librarys

In [1]:
import numpy as np
import gym
import matplotlib.pyplot as plt

Setting Up environment

In [2]:
env = gym.make('MountainCar-v0')
env.reset()

(array([-0.54019094,  0.        ], dtype=float32), {})

Env variables

In [3]:
space =  env.observation_space
actions = env.action_space
EPSILON = 1
NUM_ACTIONS = actions.n
LEARNING_RATE = 0.1
DISCOUNT_FACTOR = 0.9
NUM_EPISODES = 1000
rewards_per_episode= []
EPSILON_DECAY = 2/NUM_EPISODES


Segmenting velocity and position 

In [4]:
position = np.linspace(space.low[0], space.high[0], 20)
velocity = np.linspace(space.low[1], space.high[1], 20)

q_values = np.zeros((len(position), len(velocity), NUM_ACTIONS))

In [5]:
def select_action(position, velocity):
    if np.random.random() < EPSILON:
        action = np.random.randint(0, NUM_ACTIONS)
    else:
        action = np.argmax(q_values[position, velocity]) 
    
    return action
                

In [6]:
def train():
    state = env.reset()[0]
    state_position = np.digitize(state[0], position)
    state_velocity = np.digitize(state[1], velocity)

    goal = False

    rewards = 0

    while not goal and rewards > -1000:
        action = select_action(state_position, state_velocity)
        next_state, reward, goal, _, _ = env.step(action)
        next_state_position = np.digitize(next_state[0], position)
        next_state_velocity = np.digitize(next_state[1], velocity)
        max_q_value_next_state = np.max(q_values[next_state_position][next_state_velocity])
        q_values[state_position][state_velocity][action] += LEARNING_RATE * (reward + DISCOUNT_FACTOR * 
                                max_q_value_next_state - q_values[state_position][state_velocity][action])
        state_position = next_state_position
        state_velocity = next_state_velocity

        rewards += reward

    rewards_per_episode.append(rewards)
    EPSILON = max(0, EPSILON - EPSILON_DECAY)
    



In [7]:
for episode in range(NUM_EPISODES):
    train()

env.close()

c:\Users\admcm\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


UnboundLocalError: cannot access local variable 'EPSILON' where it is not associated with a value

In [ ]:
mean_rewards = np.zeros(NUM_EPISODES)
for t in range(NUM_EPISODES):
    mean_rewards[t] = np.mean(rewards_per_episode[max(0, t-100):(t+1)])
plt.plot(mean_rewards)